### Distribution of waiting times in SMC under a species tree model

In [37]:
import toytree
import toyplot
import ipcoal
import pandas as pd
import numpy as np

In [38]:
from ipcoal.smc.smc3 import (
    get_gene_tree_coal_intervals, 
    get_prob_gene_tree_is_unchanged_by_recomb_event,
    get_prob_gene_tree_is_unchanged_by_recomb_on_edge,
    get_prob_gene_tree_is_unchanged,
    get_expected_dist_until_gene_tree_changes,
)

### Define a species tree and sample a single genealogy

In [39]:
SPTREE = toytree.rtree.unittree(ntips=6, treeheight=1e6, seed=123)
SPTREE = SPTREE.set_node_data("Ne", {i: 5e4 for i in (0, 1, 8)}, default=1e5)
#SPTREE = SPTREE.set_node_data("Ne", default=2.5e5)
MODEL = ipcoal.Model(SPTREE, seed_trees=123)
MODEL.sim_trees(1, 1)
GTREE = toytree.tree(MODEL.df.genealogy[0])

In [40]:
def draw_trees(species_tree, gene_tree):
    """Return a drawing of species tree, gene tree, and intervals of interest.
    """   
    # draw the tree
    mtre = toytree.mtree([species_tree, gene_tree])
    canvas, axes, _ = mtre.draw(
        ts='p',
        shared_axes=True,
        scale_bar=True,
        fixed_order=SPTREE.get_tip_labels(),
        node_labels="idx",
        node_labels_style={"baseline-shift": "10px", "font-size": "11px"},
        node_sizes=6,
        height=325, width=500,
    );

    axes[0].label.text = "SPTREE"
    axes[1].label.text = "GTREE"
    for ax in axes:
        ax.hlines(
            species_tree.get_node_data("height").unique(),
            style={
                "stroke": toytree.COLORS1[1], 
                "stroke-width": 2,
                "stroke-dasharray": "2,4"},
        )
    return canvas, axes, _

### get the genetree coal events on a specified gene tree edge idx

In [45]:
-np.exp(709)

-8.218407461554972e+307

In [47]:
0.1/ 1e6

1.0000000000000001e-07

In [41]:
get_gene_tree_coal_intervals(SPTREE, GTREE, 2)

,start,stop,neff,edges,st_node,event,dist
0,0.0,250000.000000,100000.0,1,2->6,ILS,250000.000000
1,250000.0,500000.000000,100000.0,2,6->7,ILS,250000.000000
2,500000.0,696076.852632,100000.0,3,7->7,COAL-g7,196076.852632


In [28]:
%%timeit
SPTREE.set_node_data("ncoals", default=0)

282 µs ± 5.38 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [30]:
%%timeit
gene_tree = GTREE.set_node_data(
    feature="age",
    mapping=GTREE.get_node_data("height").to_dict()
)

723 µs ± 13.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [32]:
%%timeit
ncoals = {}
for node in SPTREE.treenode.traverse("postorder"):
    if not node.is_leaf():
        pass#st_tips = node.get_leaf_names()
        

9.38 µs ± 23.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [51]:
-np.exp(-1e7)

-0.0

In [16]:
draw_trees(SPTREE, GTREE);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="500.0px" height="325.0px" viewBox="0 0 500.0 325.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t335ad529e2544833b4c3394878005816"> 0 1 2 3 4 5 6 7 8 9 10 r0 r1 r2 r3 r4 r5 0 400000 800000 1200000 SPTREE 0 1 2 3 4 5 6 7 8 9 10 r4 r3 r2 r5 r1 r0 0 400000 800000 1200000 GTREE

### get the prob. gene tree does *not* change given recomb at edge x time

In [17]:
get_prob_gene_tree_is_unchanged_by_recomb_event(SPTREE, GTREE, (3, 500_000))

0.9572377923299176

In [18]:
def plot_edge_recomb_probs(species_tree, gene_tree, idx, **kwargs):
    """plots prob gtree is unchanged for recomb positions along the edge"""
    node = gene_tree.idx_dict[idx]
    xpos = np.linspace(node.height, node.up.height, 50)
    probs = []
    for time in xpos:
        prob = get_prob_gene_tree_is_unchanged_by_recomb_event(
            species_tree, gene_tree, (idx, time)
        )
        probs.append(prob)
    
    # optional styling kwargs
    style = {
        'width': 350, 
        'height': 300, 
        'xlabel': "position of recomb. on edge", 
        'ylabel': "prob. gene tree is unchanged.",
        'style': {"stroke-width": 3},
        'ymin': 0,
    }
    style.update(kwargs)
    
    # plot the prob. of unchanged given position of recomb.
    return toyplot.plot(xpos, probs, **style);

In [19]:
plot_edge_recomb_probs(SPTREE, GTREE, 0);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="300.0px" viewBox="0 0 350.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t86e8578d68a34644a80071a43115cb08"> <path d="M 50.0 50.915280107162864 L 55.10204081632653 51.036484576056225 L 60.20408163265306 51.173739348200755 L 65.3061224489796 51.329169858712895 L 70.40816326530613 51.50518299997269 L 75.51020408163266 51.70450439314109 L 80.61224489795919 51.930220595296376 L 85.71428571428572 52.18582689578195 L 90.81632653061226 52.475281441906986 L 95.91836734693878 52.803066532154276 L 101.02040816326532 53.174258026040945 L 106.12244897959184 53.594603945465956 L 111.22448979591836 54.07061348471258 L 116.32653061224491 54.609657807454525 L 121.42857142857142 55.220084191640396 L 126.53061224489797 55.91134528982777 L 131.6326530612245 56.694145506604876 L 136.73469387755102 57.5806067598031 L 141.83673469387756 58.584456192365835 L 146.9387755102041 59.72123874165478 L 152.04081632653063 61.008557857898005 L 157.1428571428571 62.46634809938416 L 162.24489795918367 64.11718382562906 L 167.3469387755102 65.9866287687288 L 172.44897959183672 68.10363189612704 L 177.55102040816328 70.50097569486067 L 182.65306122448982 73.21578381911205 L 187.75510204081633 76.2900959621575 L 192.85714285714283 79.77151885479113 L 197.9591836734694 83.71396347114917 L 203.06122448979593 88.17847985780752 L 208.16326530612247 93.23420251375538 L 213.265306122449 98.95942096076307 L 218.3673469387755 105.44279208227724 L 223.46938775510202 112.78471300430826 L 228.57142857142856 121.09887577782035 L 233.67346938775512 130.51402793739075 L 238.77551020408166 141.1759661989647 L 243.8775510204082 153.2497941697705 L 248.9795918367347 166.92247903177176 L 254.08163265306126 182.40574678973707 L 259.18367346938777 183.46004531981967 L 264.2857142857143 183.51734449216573 L 269.38775510204084 183.60055435569885 L 274.48979591836735 183.7213917274392 L 279.59183673469386 183.89687177017848 L 284.6938775510204 184.15170390494188 L 289.7959183673469 184.52177115529307 L 294.89795918367344 185.05918285146583 L 300.0 185.83961216778386" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:3"> 0 200000 400000 600000 position of recomb. on edge 0.0 0.5 1.0 prob. gene tree is unchanged.

In [60]:
from decimal import Decimal

In [69]:
a = Decimal(700.000)
b = Decimal(800.322)

In [70]:
np.exp(b) - np.exp(a)

Decimal('3.762082765615092907558784684498222459674255591664278458601422654430489014422676468461548300523140101E+347')

In [71]:
import decimal
decimal.getcontext().prec = 100

In [75]:
np.exp(np.array([900], dtype=np.float128))

array([7.32881422e+390], dtype=float128)

In [79]:
np.exp(np.float128(9000))

4.4703047331654426408e+3908

In [ ]:
WARNING | smc_pat.py   | **** 0-1
first_term=500000.0
second_term_before=30793.83379790111
first_expr=2.4903429574618414
second_expr=-83808.08494617493
branch_sum=291289.1258759295
--------------

WARNING | smc_pat.py   | **** 1-1
first_term=72909.02135234904
second_term_before=0
first_expr=176.7816208899733
second_expr=-202.6625475142001
branch_sum=37082.00770909751
--------------

### Get prob gene tree is unchanged given recomb on edge

In [10]:
# integration over the unif prob of recomb anywhere on edge
get_prob_gene_tree_is_unchanged_by_recomb_on_edge(SPTREE, GTREE, 3)

0.7394815308146538

### Get prob gene tree is unchanged given recomb on *any edge*

In [11]:
get_prob_gene_tree_is_unchanged(SPTREE, GTREE)

0.6350486982672543

### Get expected (spatial) waiting time until gene tree changes 

In [12]:
get_expected_dist_until_gene_tree_changes(SPTREE, GTREE, 1e-8)

49.912035634775854

### Compare expected distance to simulations

In [34]:
# model uses the variable Ne values assigned to SPTREE nodes
model = ipcoal.Model(SPTREE, recomb=1e-9, seed_trees=123)
model.sim_loci(nloci=1, nsites=1e7)

In [35]:
# how long did each genealogy span (nbps) column
observed = model.df.nbps

In [36]:
# distribute computation of 'expected' in parallel
from concurrent.futures import ProcessPoolExecutor
with ProcessPoolExecutor(max_workers=6) as pool:
    rasyncs = {}
    for idx in model.df.index:
        gtree = toytree.tree(model.df.genealogy[idx])
        args = (SPTREE, gtree, 1e-9)
        rasyncs[idx] = pool.submit(get_expected_dist_until_gene_tree_changes, *args)

# collect results
expected_waiting_times = np.array([rasyncs[idx].result() for idx in rasyncs])

In [53]:
# simulation-based expectations
observed.mean(), observed.std()

NameError: name 'observed' is not defined

In [81]:
import scipy.stats

In [89]:
scipy.stats.expon(657)

In [93]:
# now sample random lengths from the expected waiting times
expected_len_dists = np.random.exponential(expected_waiting_times)
expected_len_dists.mean(), expected_len_dists.std()

NameError: name 'expected_waiting_times' is not defined

In [60]:
# now sample random lengths from the expected waiting times
expected_len_dists = np.random.exponential(expected_waiting_times)
expected_len_dists.mean(), expected_len_dists.std()

(525.697274589227, 537.201712529203)

In [122]:
sorted(SPTREE.get_node_data("height")[SPTREE.ntips:] / (2 * 1e5))

[1.25, 2.5, 3.75, 3.75, 5.0]

In [121]:
toytree.rtree.bdtree(10).get_node_data("height").sort_values()

9     0.000000e+00
8     0.000000e+00
6     0.000000e+00
5     0.000000e+00
4     0.000000e+00
0     4.440892e-16
2     4.440892e-16
1     4.440892e-16
3     4.440892e-16
7     8.881784e-16
10    1.350746e-02
11    3.906704e-02
12    6.948640e-01
13    1.145003e+00
14    1.526557e+00
17    1.651152e+00
15    1.836662e+00
16    2.067886e+00
18    2.458722e+00
Name: height, dtype: float64

In [123]:
from math import comb

In [125]:
comb(4, 2)

6

In [106]:
%%timeit
comb(10, 3)

13 µs ± 1.14 µs per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [103]:
from scipy.special import comb

In [105]:
%%timeit
comb(10, 3)

12.2 µs ± 27.2 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [95]:
canvas = toyplot.Canvas(width=600, height=300)
ax0 = canvas.cartesian(grid=(1, 2, 0), xmax=3000, label="simulations")
ax1 = canvas.cartesian(grid=(1, 2, 1), xmax=3000, label="expected")
ax0.bars(np.histogram(observed, bins=np.linspace(0, 3000, 20)));
ax1.bars(np.histogram(expected_len_dists, bins=np.linspace(0, 3000, 20)));

NameError: name 'observed' is not defined

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="600.0px" height="300.0px" viewBox="0 0 600.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tceae750c3f574c239a0a0c5a1bc61974"> 0 1000 2000 3000 -0.5 0.0 0.5 simulations 0 1000 2000 3000 -0.5 0.0 0.5 expected